## Why Apache Spark for Big Data?

1. Easy to use. Provides high-level API that focuses on the content of the computation.
2. Fast, enabling interactive use and complex algorithms.
3. General engine. Combines multiple types of computations (SQL queries, text processing, and ML)

## Chapter 1: Introduction to Data Analysis with Spark

### What is Apache Spark?

1. Apache Spark is a cluster computing platform designed to be fast and general-purpose.
2. Ability to run computation in memory.
3. More efficient than MapReduce for complex applications.
4. Integrate closely with other Big Data tools.

## A Unified Stack

1. Spark Core - Task scheduling, memory management, RDD API
2. Spark SQL - Structured data
3. Spark streaming - Live stream of data in real time
4. MLlib machine learning
5. GraphX graph processing
6. Cluster Managers - Standalone, YARN, Mesos

### Users of Spark

1. Data Scientist
2. Engineer